In [13]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
from rapidfuzz import fuzz
from tqdm import tqdm

In [14]:
df = pd.read_pickle("data_sources/filtered/ntsb.pkl")
df.sample(10)

,Vehicles.VehicleNumber,Vehicles.DamageLevel,Vehicles.ExplosionType,Vehicles.FireType,Vehicles.SerialNumber,Vehicles.Make,Vehicles.Model,Vehicles.NumberOfEngines,Vehicles.RegistrationNumber,Vehicles.FlightOperationType,...,Country,EventDate,State,Agency,EventType,AirportId,AirportName,Latitude,Longitude,TotalInjuryCount
15932,1,substantial,none,none,t18208439,cessna,t182t,1,n357tg,pers,...,usa,2008-06-29 13:40:00,nj,ntsb,acc,n07,lincoln park airport,40.947498,-74.314445,0
2259,1,substantial,none,none,299,socata,tbm-700 c2,1,n48um,pers,...,usa,2021-07-27 12:05:00,sc,ntsb,acc,lro,mt pleasant rgnl-faison fld,32.903011,-79.784190,0
11348,1,substantial,none,none,3290,mooney,m20c,1,n2610w,pers,...,usa,2012-05-19 12:20:00,fl,ntsb,acc,hwo,north perry airport,26.001111,-80.240554,1
22156,1,substantial,none,none,15285671,cessna,152,1,n94445,inst,...,usa,2003-10-22 11:30:00,fl,ntsb,acc,None,None,29.230581,-81.459403,2
11855,1,minor,none,in-flight,23719,boeing,747-451,4,n661us,None,...,usa,2011-10-23 16:00:00,mi,ntsb,inc,dtw,detroit international airport,42.349647,-83.059921,0
3969,1,none,none,none,8118,airbus,a320,2,n328fr,None,...,usa,2019-08-27 07:50:00,co,ntsb,acc,kden,denver international airport,39.861667,-104.673057,1
7118,2,NaN,unknown,unknown,unknown,unknown,unknown,0,unknown,None,...,usa,2016-07-17 10:30:00,id,ntsb,acc,None,None,43.840278,-116.501388,0
1348,1,substantial,none,none,502-0171,air tractor inc,at-502,1,n1540s,aapl,...,usa,2022-07-21 19:30:00,mn,ntsb,acc,n/a,None,47.930686,-97.012351,0
13195,1,substantial,none,none,1259,dehavilland,beaver dhc-2 mk.1,1,n24150,pers,...,usa,2010-09-19 15:30:00,wa,ntsb,acc,w36,will rogers wiley post,47.518054,-122.218055,0
9355,1,minor,none,none,525b0014,cessna,525,2,n300et,pers,...,usa,2014-04-26 11:30:00,fl,ntsb,inc,7fl6,spruce creek,29.076389,-81.052223,0


In [24]:
ntsb_og = df.copy()
ntsb_copied = df.copy()
ntsb_copied = ntsb_copied.rename(columns=
    {"EventDate": "Date", 
    "NtsbNumber": "ID", 
    "State": "Location"}) # for schema matching 

seed = 10
np.random.seed(seed)

n = len(ntsb_copied)

# select 60% indices for later use
n_forty = int(np.floor(0.6 * n))
random_indices = np.random.choice(ntsb_copied.index, n_forty, replace=False)
print(f"random_indices count (60%): {len(random_indices)}")  # Should be ~0.6 * n

# select half of these indices to assign NaN in 'ID'
n_missing = int(np.floor(0.5 * n_forty))
missing_indices = np.random.choice(random_indices, n_missing, replace=False)

# assign NaN only to these missing_indices, in original to then match
ntsb_og.loc[missing_indices, "State"] = np.nan  # for slot filling

print(f"Number of NaNs assigned: {len(missing_indices)}")

# count total NaNs in 'ID' (including existing NaNs)
total_nans = ntsb_og['State'].isna().sum()
print(f"Total NaNs in 'State' column: {total_nans}")

conflict_indices = np.setdiff1d(random_indices, missing_indices)

# transform strings for conflict resolution
for index, row in ntsb_copied.iterrows():
    if index in conflict_indices:
        airport = row["AirportName"]
        if pd.notna(airport):
            result = ' '.join([word[0] + '.' for word in airport.split()])
            ntsb_copied.loc[index, "AirportName"] = result

ntsb_og.to_pickle("data_sources/to_fuse/ntsb_og.pkl")
ntsb_copied.to_pickle("data_sources/to_fuse/ntsb_copied.pkl")

random_indices count (60%): 14041
Number of NaNs assigned: 7020
Total NaNs in 'State' column: 7050


### Fuse NTSB with its dupe for strategy implemenation

In [28]:
tqdm.pandas()

# Load datasets
ntsb_og = pd.read_pickle('data_sources/to_fuse/ntsb_og.pkl')
ntsb_copied = pd.read_pickle('data_sources/to_fuse/ntsb_copied.pkl')  # adjust if needed

# NaN count BEFORE fusion
id_nans_before = ntsb_og['State'].isna().sum()
print(f"NaN in 'State' (before fusion): {id_nans_before}")

# Count abbreviated AirportName BEFORE fusion (optional)
abbrev_pattern = r'^([a-zA-Z]\.\s*)+$'
airport_abbrev_before = ntsb_copied['AirportName'].fillna('').str.match(abbrev_pattern).sum()
print(f"Abbreviated AirportName (before fusion): {airport_abbrev_before}")

# Normalize column names in ntsb_copied
ntsb_copied = ntsb_copied.rename(columns={
    "Date": "EventDate", 
    "ID": "NtsbNumber", 
    "Location": "State"
})

# Drop NtsbNumber
ntsb_og = ntsb_og.drop(columns=['NtsbNumber'], errors='ignore')
ntsb_copied = ntsb_copied.drop(columns=['NtsbNumber'], errors='ignore')

# Helper: fuzzy similarity for airport names
def fuzzy_match_airport(row, df_ref):
    candidates = df_ref[
        (df_ref['EventDate'] == row['EventDate']) &
        (df_ref['Vehicles.RegistrationNumber'] == row['Vehicles.RegistrationNumber']) &
        (df_ref['Vehicles.SerialNumber'] == row['Vehicles.SerialNumber'])
    ]
    if candidates.empty or pd.isna(row['AirportName']):
        return None
    best = candidates['AirportName'].dropna().apply(lambda x: fuzz.partial_ratio(x, row['AirportName']))
    if not best.empty and best.max() >= 80:
        return candidates.iloc[best.idxmax()]
    return None

# Merge logic
fused_rows = []
unmatched_rows = []

for _, row in tqdm(ntsb_copied.iterrows(), total=len(ntsb_copied), desc="Merging entries"):
    match = ntsb_og[
        (ntsb_og['EventDate'] == row['EventDate']) &
        (ntsb_og['Vehicles.RegistrationNumber'] == row['Vehicles.RegistrationNumber']) &
        (ntsb_og['Vehicles.SerialNumber'] == row['Vehicles.SerialNumber'])
    ]

    if match.empty:
        unmatched_rows.append(row)
        continue

    merged = match.iloc[0].copy()

    # Slot Filling: if merged[col] is NA and row[col] is not, use row[col]
    for col in ntsb_copied.columns:
        if col in merged.index and pd.isna(merged[col]) and pd.notna(row[col]):
            merged[col] = row[col]
    
    # Conflict resolution: prioritize df unless row[col] is clearly more complete
    if pd.notna(row['AirportName']) and pd.notna(merged['AirportName']):
        if len(row['AirportName']) > len(merged['AirportName']):  # assume longer name is better
            merged['AirportName'] = row['AirportName']

    fused_rows.append(merged)

# Construct final fused dataframe
fused_df = pd.DataFrame(fused_rows)

# Reattach unmatched rows (optional)
fused_df = pd.concat([fused_df, pd.DataFrame(unmatched_rows)], ignore_index=True)

before_dedup = len(fused_df)
# Deduplicate
fused_df = fused_df.drop_duplicates(subset=[
    'EventDate', 'Vehicles.SerialNumber', 'Vehicles.RegistrationNumber'
])

# Save final fused result
fused_df.to_pickle('data_sources/fused/ntsb_fused.pkl')
fused_df.to_csv('data_sources/fused/ntsb_fused.csv', index=False)

print("Final fused dataset saved:")
print(" • accident_weather_fused_final.pkl")
print(" • accident_weather_fused_final.csv")
print(f"Final row count: {len(fused_df)} (original: {len(df)}, added: {len(unmatched_rows)})")

# NaN count AFTER fusion
id_nans_after = fused_df['State'].isna().sum()
print(f"NaN in 'State' (after fusion): {id_nans_after}")
filled_count = id_nans_before - id_nans_after
nan_percent = filled_count / id_nans_before * 100
print(f"'State' values filled during fusion: {filled_count} ({nan_percent:.2f}%)")

# Deduplication stats
after_dedup = len(fused_df)
dedup_removed = before_dedup - after_dedup
dedup_percent = (dedup_removed / before_dedup) * 100
print(f"Deduplication removed: {dedup_removed} rows ({dedup_percent:.2f}%)")

# Count abbreviated AirportName AFTER fusion
abbrev_pattern = r'^([a-zA-Z]\.\s*)+$'
airport_abbrev_after = fused_df['AirportName'].fillna('').str.match(abbrev_pattern).sum()
print(f"Abbreviated AirportName (after fusion): {airport_abbrev_after}")
resolved_count = airport_abbrev_before - airport_abbrev_after
resolved_percent = resolved_count / airport_abbrev_before * 100
print(f"Resolved AirportName values: {resolved_count} ({resolved_percent:.2f}%)")

NaN in 'State' (before fusion): 7050
Abbreviated AirportName (before fusion): 5125


Merging entries: 100%|██████████| 23403/23403 [01:08<00:00, 340.26it/s]


Final fused dataset saved:
 • accident_weather_fused_final.pkl
 • accident_weather_fused_final.csv
Final row count: 23400 (original: 23403, added: 120)
NaN in 'State' (after fusion): 47
'State' values filled during fusion: 7003 (99.33%)
Deduplication removed: 3 rows (0.01%)
Abbreviated AirportName (after fusion): 20
Resolved AirportName values: 5105 (99.61%)


## Weather Data Fusion

In [10]:
# spatial & temporal thresholds
LAT_LON_EPS   = 0.10       # ≈ 11 km at mid-latitudes
MAX_TIME_DIFF = pd.Timedelta('3h')   # reject candidates > 3 h away

# --- 1. Load data -------------------------------------------------------------
ntsb_path    = Path("data_sources/filtered/ntsb.pkl")
weather_path = Path("data_sources/filtered/weather.pkl")

ntsb    = pd.read_pickle(ntsb_path)
weather = pd.read_pickle(weather_path)

# ensure correct dtypes
ntsb["EventDate"] = pd.to_datetime(ntsb["EventDate"], errors="coerce")
weather["time"]   = pd.to_datetime(weather["time"],   errors="coerce")

# --- 2. Blocking on event *date* ---------------------------------------------
ntsb["event_day"]    = ntsb["EventDate"].dt.date
weather["weather_day"] = weather["time"].dt.date

weather_by_day = {d: w.reset_index(drop=True)
                  for d, w in weather.groupby("weather_day")}

# --- 3. Similarity matching & temporal precedence -----------------------------
best_rows = []       # stores best-matching weather rows (or None)

for _, acc in ntsb.iterrows():
    day_candidates = weather_by_day.get(acc["event_day"], pd.DataFrame())
    if day_candidates.empty:
        best_rows.append(None); continue
    
    # coarse spatial filter  |lat/lon diff| < LAT_LON_EPS
    spatial = day_candidates[
        (day_candidates["time"].notna()) &
        (day_candidates["AccidentID"].notna()) &       # keeps malformed rows out
        (day_candidates["AccidentID"].str.contains('_'))  # quick sanity
    ].copy()

    spatial = spatial[
        (np.abs(spatial["AccidentID"].str.split('_').str[-2].astype(float) - acc["Latitude" ] ) < LAT_LON_EPS) &
        (np.abs(spatial["AccidentID"].str.split('_').str[-1].astype(float) - acc["Longitude"]) < LAT_LON_EPS)
    ]

    if spatial.empty:
        best_rows.append(None); continue
    
    # temporal distance to the accident moment
    spatial["time_diff"] = (spatial["time"] - acc["EventDate"]).abs()
    
    # keep the closest hour that is still within MAX_TIME_DIFF
    spatial = spatial[spatial["time_diff"] <= MAX_TIME_DIFF]
    
    best_rows.append(spatial.nsmallest(1, "time_diff").iloc[0] if not spatial.empty else None)

# --- 4. Assemble the fused dataset -------------------------------------------
weather_match_df = pd.DataFrame.from_records(
    [row.to_dict() if row is not None else {}          # convert None into an empty dict {}
     for row in best_rows],
    index=ntsb.index                                   # keeps row-alignment
)

accident_weather = pd.concat(
    [ntsb.reset_index(drop=True),
     weather_match_df.add_prefix("wx_")],              # prefix to avoid clashes
    axis=1
)

# --- 5. Quick diagnostics -----------------------------------------------------
total_accidents = len(ntsb)
matched         = accident_weather["wx_time"].notna().sum()
print(f"Matched {matched} of {total_accidents} accidents "
      f"({matched / total_accidents:.1%})")

if matched:
    print("\nTime difference (min) for matched rows:")
    print((accident_weather.loc[accident_weather.wx_time.notna(), "wx_time_diff"]
           .dt.total_seconds().div(60)
           .describe().round(2)))

    print("\nSpatial deltas (deg lat/lon) for matched rows:")
    lat_delta = np.abs(accident_weather["Latitude"] - accident_weather["wx_AccidentID"]
                       .str.split('_').str[-2].astype(float))
    lon_delta = np.abs(accident_weather["Longitude"] - accident_weather["wx_AccidentID"]
                       .str.split('_').str[-1].astype(float))
    print(pd.concat({"lat": lat_delta, "lon": lon_delta}, axis=1).describe().round(4))

accident_weather.drop(columns=["event_day","wx_AccidentID","wx_weather_day"], errors='ignore', inplace=True)
accident_weather.to_pickle("data_sources/fused/accident_weather.pkl")
accident_weather.to_csv("data_sources/fused/accident_weather.csv", index=False)

KeyboardInterrupt: 

## Matched Aircraft Data Fusion

In [ ]:
# Load the datasets
accident_weather_path = 'data_sources/fused/accident_weather.pkl'
matched_results_path = 'data_sources/binding/matched_results.csv'

accident_weather_df = pd.read_pickle(accident_weather_path)
matched_results_df = pd.read_csv(matched_results_path)


def clean_text(s):
    """ Normalizzazione del testo: rimozione di caratteri speciali, lowercase e spazi extra. """
    return re.sub(r'\W+', ' ', str(s)).lower().strip()

# Pulizia dei dati
accident_weather_df['Vehicles.Model'] = accident_weather_df['Vehicles.Model'].apply(clean_text)
accident_weather_df['Vehicles.Make'] = accident_weather_df['Vehicles.Make'].apply(clean_text)

# Normalize casing for matching
matched_results_df['NtsbNumber'] = matched_results_df['NtsbNumber'].str.lower()
matched_results_df['EventDate'] = pd.to_datetime(matched_results_df['EventDate'], errors='coerce')
matched_results_df['Vehicles.SerialNumber'] = matched_results_df['Vehicles.SerialNumber'].str.lower()
matched_results_df['Vehicles.RegistrationNumber'] = matched_results_df['Vehicles.RegistrationNumber'].str.lower()
matched_results_df['Vehicles.Make'] = matched_results_df['Vehicles.Make'].str.lower()
matched_results_df['Vehicles.Model'] = matched_results_df['Vehicles.Model'].str.lower()

matched_results_df.drop(columns=["JW_Score","LEV_Score","Jac_Score","SimilarityScore","Matched_Aircraft_Model"], errors='ignore', inplace=True)


accident_weather_df['NtsbNumber'] = accident_weather_df['NtsbNumber'].str.lower()
accident_weather_df['EventDate'] = pd.to_datetime(accident_weather_df['EventDate'], errors='coerce')
accident_weather_df['Vehicles.SerialNumber'] = accident_weather_df['Vehicles.SerialNumber'].astype(str).str.lower()
accident_weather_df['Vehicles.RegistrationNumber'] = accident_weather_df['Vehicles.RegistrationNumber'].astype(str).str.lower()
accident_weather_df['Vehicles.Make'] = accident_weather_df['Vehicles.Make'].astype(str).str.lower()
accident_weather_df['Vehicles.Model'] = accident_weather_df['Vehicles.Model'].astype(str).str.lower()

accident_weather_df.drop(columns=["Vehicles.VehicleNumber"], errors='ignore', inplace=True)
accident_weather_df.rename(columns={"wx_time": "weather_time"}, inplace=True)

for key in accident_weather_df.columns:
    if key.startswith('wx_'):
        accident_weather_df.rename(columns={key: key[3:]}, inplace=True)

# Define the merge keys
merge_keys = ['NtsbNumber','EventDate','Vehicles.SerialNumber', 'Vehicles.RegistrationNumber', 'Vehicles.Make', 'Vehicles.Model']

# Perform the merge
fused_df = accident_weather_df.merge(
    matched_results_df,
    how='left',
    left_on=merge_keys,
    right_on=merge_keys
)

# Drop the duplicate matching columns from the right
for key in merge_keys:
    fused_df.drop(columns=[f"{key}_y"], errors='ignore', inplace=True)
    fused_df.rename(columns={f"{key}_x": key}, inplace=True)

# Save the resulting dataframe
fused_df.to_pickle('data_sources/fused/accident_weather_enriched.pkl')
fused_df.to_csv("data_sources/fused/accident_weather_enriched.csv", index=False)

# Compute matching stats
total_records = len(accident_weather_df)
matched_records = fused_df['Matched_Aircraft_Model'].notna().sum()
unmatched_records = total_records - matched_records
match_percentage = (matched_records / total_records) * 100

# Print statistics
print("Fusion complete. Enriched dataset saved to: data_sources/fused/accident_weather_enriched.pkl")
print("\n--- Matching Statistics ---")
print(f"Total records in original dataset: {total_records}")
print(f"Total records matched with binding CSV: {matched_records}")
print(f"Total unmatched records: {unmatched_records}")
print(f"Match percentage: {match_percentage:.2f}%")


Fusion complete. Enriched dataset saved to: data_sources/fused/accident_weather_enriched.pkl

--- Matching Statistics ---
Total records in original dataset: 23403
Total records matched with binding CSV: 4962
Total unmatched records: 18441
Match percentage: 21.20%


In [ ]:
fused_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23403 entries, 0 to 23402
Data columns (total 49 columns):
 #   Column                        Non-Null Count  Dtype          
---  ------                        --------------  -----          
 0   Vehicles.DamageLevel          23400 non-null  category       
 1   Vehicles.ExplosionType        21880 non-null  category       
 2   Vehicles.FireType             23321 non-null  category       
 3   Vehicles.SerialNumber         23403 non-null  object         
 4   Vehicles.Make                 23403 non-null  object         
 5   Vehicles.Model                23403 non-null  object         
 6   Vehicles.NumberOfEngines      23403 non-null  int64          
 7   Vehicles.RegistrationNumber   23403 non-null  object         
 8   Vehicles.FlightOperationType  21593 non-null  object         
 9   Vehicles.OperatorName         11290 non-null  object         
 10  Oid                           23403 non-null  object         
 11  MKey           

## fixing issue between `engine_count` and `Vehicles.NumberOfEngines`

In [ ]:
# Load the dataset
df_path = 'data_sources/fused/accident_weather_enriched.pkl'
df = pd.read_pickle(df_path)

# Convert columns to nullable integers
engine_count_int = df['engine_count'].astype('Int64')
vehicle_engines = df['Vehicles.NumberOfEngines'].astype('Int64')

# Rule 1: Fill NaNs in Vehicles.NumberOfEngines with engine_count
df['Vehicles.NumberOfEngines'] = vehicle_engines.combine_first(engine_count_int)

# Rule 2: If Vehicles.NumberOfEngines == 0 and engine_count > 0 → trust engine_count
mask_replace_zero = (
    (df['Vehicles.NumberOfEngines'] == 0) &
    (engine_count_int > 0)
)
df.loc[mask_replace_zero, 'Vehicles.NumberOfEngines'] = engine_count_int[mask_replace_zero]

# Rule 3: Overwrite in case of real conflict (≠ 0 and ≠ each other)
conflict_mask = (
    engine_count_int.notna() &
    df['Vehicles.NumberOfEngines'].notna() &
    (df['Vehicles.NumberOfEngines'] != engine_count_int) &
    (df['Vehicles.NumberOfEngines'] != 0) &
    (engine_count_int != 0)
)
df.loc[conflict_mask, 'Vehicles.NumberOfEngines'] = engine_count_int[conflict_mask]

# Drop auxiliary column
df.drop(columns=['engine_count'], inplace=True)

# Save cleaned and final dataset
final_pkl_path = 'data_sources/fused/accident_weather_final.pkl'
final_csv_path = 'data_sources/fused/accident_weather_final.csv'

df.to_pickle(final_pkl_path)
df.to_csv(final_csv_path, index=False)

print(f"✅ Fusion complete. Cleaned dataset saved to:\n  • {final_pkl_path}\n  • {final_csv_path}")
print(f"🔄 {conflict_mask.sum()} engine count conflicts were resolved by trusting the 'engine_count' value.")

✅ Fusion complete. Cleaned dataset saved to:
  • data_sources/fused/accident_weather_final.pkl
  • data_sources/fused/accident_weather_final.csv
🔄 8 engine count conflicts were resolved by trusting the 'engine_count' value.
